<b>Написать приложение, которое собирает основные новости с сайтов mail.ru, lenta.ru, yandex-новости. Для парсинга использовать XPath. Структура данных должна содержать:</b><br>
название источника;<br>
наименование новости;<br>
ссылку на новость;<br>
дата публикации.

In [120]:
from pprint import pprint
from lxml import html
import requests
import datetime
import pandas as pd

In [121]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'}

In [122]:
def get_mail():
    
    data = []
    
    site = 'https://mail.ru/'
    response = requests.get(site, headers = header)
    root = html.fromstring(response.text)
    news = root.xpath("//li[contains(@data-testid, 'general-news-item')]")
    i=0
    for item in news:
        if i == 0:
            title = item.xpath(".//a/*/p/text()")
        else:
            title = item.xpath(".//a[not(contains(@class, 'news-item__label'))]/text()")

        link = item.xpath(".//a[not(contains(@class, 'news-item__label'))]/@href")
        now = datetime.datetime.now()
        published = now.strftime("%H:%M, %d %B %Y")
        
        i += 1

        this = ({'source': site, 'title': title[0].replace('\t', ' '), 'link': link[0], 'published': published}) 
        data.append(this)
        
    return data

In [131]:
def get_lenta():
    
    data = []
    
    site = 'https://lenta.ru'
    response = requests.get(site, headers = header)
    root = html.fromstring(response.text)
    news = root.xpath("//section[contains(@class, 'b-top7-for-main')]//div[contains(@class, 'item')]")
    for item in news:
        title = item.xpath(".//a[not(contains(@class, 'b-favorite__item'))]/text()")
        link = item.xpath(".//a[not(contains(@class, 'b-favorite__item')) and not(contains(@class, 'topic-title-pic__link'))]/@href")
        published = item.xpath(".//time/@datetime")
        this = ({'source': site, 'title': title[0].replace('\xa0', ' '), 'link': link[0] if link[0].startswith("http") else site+link[0], 'published': published[0]}) 
        data.append(this)
        
    return data

In [124]:
def get_yandex():
    
    data = []
    
    site = 'https://yandex.ru/news'
    response = requests.get(site, headers = header)
    root = html.fromstring(response.text)
    news = root.xpath("//div[contains(@class, 'news-top-flexible-stories')]//article")
    for item in news:
        title = item.xpath(".//h2[contains(@class, 'mg-card__title')]/text()")
        link = item.xpath(".//a[not(contains(@class, 'mg-card__source-link'))]/@href")
        published = item.xpath(".//span[contains(@class, 'mg-card-source__time')]/text()")
        now = datetime.datetime.now()
        published = f'{published[0]}, {now.strftime("%d %B %Y")}'
        
        this = ({'source': site, 'title': title[0].replace('\xa0', ' '), 'link': link[0], 'published': published}) 
        data.append(this)
    
    return data

In [132]:
mailru = get_mail()
lenta = get_lenta()
yandex = get_yandex()
all = mailru + lenta + yandex
df = pd.DataFrame(all)

In [133]:
def make_clickable(val):
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)
df.style.format({'link': make_clickable})

,source,title,link,published
0,https://mail.ru/,В США заявили о намерении прекратить гонки вооружений,https://news.mail.ru/politics/48338566/?frommail=1,"23:38, 13 October 2021"
1,https://mail.ru/,В Норвегии лучник застрелил несколько человек,https://news.mail.ru/incident/48338339/?frommail=1,"23:38, 13 October 2021"
2,https://mail.ru/,"Названы симптомы COVID, при которых нужно срочно вызывать врача",https://news.mail.ru/society/48326667/?frommail=1,"23:38, 13 October 2021"
3,https://mail.ru/,"Обнаружены антитела, защищающие от всех штаммов COVID-19",https://news.mail.ru/society/48330576/?frommail=1,"23:38, 13 October 2021"
4,https://mail.ru/,Жительница Турции признана самой высокой женщиной в мире (фото),https://news.mail.ru/society/48335674/?cr=123&frommail=10,"23:38, 13 October 2021"
5,https://mail.ru/,Фото окуней во время нереста стало лучшим снимком года,https://news.mail.ru/society/48332099/?frommail=10,"23:38, 13 October 2021"
6,https://mail.ru/,"Черногория назвала россиян, получивших «золотые паспорта»",https://news.mail.ru/economics/48327357/?frommail=1,"23:38, 13 October 2021"
7,https://mail.ru/,Журналист Дмитрий Гордон призвал Украину не вступать в Евросоюз,https://news.mail.ru/politics/48335774/?frommail=1,"23:38, 13 October 2021"
8,https://mail.ru/,Фото дня: как добывают розовую соль в Крыму,https://news.mail.ru/society/44226108/?frommail=10,"23:38, 13 October 2021"
9,https://mail.ru/,"Спецназ КНДР показал Ким Чен Ыну, на что способен (видео)",https://news.mail.ru/politics/48334311/?frommail=10,"23:38, 13 October 2021"
